# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

C:\Users\abooth\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\abooth\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\abooth\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("output_data/cities_booth.csv")
df.head()

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed
0,belushya guba,71.537698,52.342734,Clouds,19.02,90,1012,3.38
1,zhangye,38.936513,100.455633,Clear,38.43,37,1016,5.26
2,saint-philippe,45.358450,-73.479005,Clear,42.69,46,1038,13.80
3,hilo,19.707373,-155.081580,Rain,76.28,82,1021,6.91
4,rikitea,-23.122954,-134.968014,Clouds,75.54,89,1015,14.25


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# get data
data = df.loc[:, ["latitude", "longitude"]]

# Create BASE, init the graph
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='SATELLITE')

# init the overlay layers
heat = gmaps.heatmap_layer(data, weights=df.humidity)
fig.add_layer(heat)

# show the plot
fig

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
mask = (df.wind_speed < 10) & (df.temperature < 80) & (df.temperature > 70) & (df.weather == "Clear")
hotel_df = df.loc[mask].reset_index(drop=True)
hotel_df

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed
0,biltine,14.525174,20.928701,Clear,78.44,20,1011,8.59
1,bargal,11.286823,51.077950,Clear,78.40,84,1013,5.12
2,cayenne,4.937114,-52.325831,Clear,75.22,94,1012,1.14
3,tsihombe,-25.318547,45.484057,Clear,73.17,86,1012,2.59
4,keshod,21.301130,70.252402,Clear,72.12,62,1013,7.18
5,nelson bay,-32.721814,152.144089,Clear,70.48,66,1021,1.99
6,lahij,13.057782,44.883614,Clear,76.03,80,1013,7.90
7,riyadh,24.638916,46.716010,Clear,73.78,21,1012,5.59
8,concepcion del oro,24.613705,-101.418787,Clear,72.72,36,1015,6.89
9,arlit,18.739079,7.393331,Clear,73.74,17,1015,9.26


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotels = []

for indx, row in hotel_df.iterrows():
    lat = row.latitude
    lng = row.longitude

    # geocoordinates
    target_coordinates = f"{lat}, {lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    data = response.json()
    
    if (len(data["results"]) > 0):
        hotel = data["results"][0]["name"]
        address = data["results"][0]["vicinity"]
        rating = data["results"][0]["rating"]
        user_ratings_total = data["results"][0]["user_ratings_total"]
    else:
        hotel = None
        address = None
        rating = np.nan
        user_ratings_total = np.nan
        
    hotel = {"hotel" : hotel,
            "address": address,
            "rating": rating,
            "user_ratings_total": user_ratings_total}
    
    hotels.append(hotel)

In [18]:
df2 = pd.DataFrame(hotels)
df2.head()

,hotel,address,rating,user_ratings_total
0,Hôtel Trois Étoiles de Biltine,"GWH7+2R8, Biltine",3.0,2.0
1,Maka Hotel,"73QH+9F5, Bargaal",3.7,9.0
2,Grand Hotel Montabo,Chemin St Hilaire - rue de Montabo,3.6,320.0
3,Littoral Hôtel,Tsihombe Tsihombe Androy,5.0,1.0
4,"Hotel Shree Inn, KESHOD","Junagadh Road, near,iocl petrol pump, Keshod",3.7,332.0


In [20]:
hotel_full = pd.merge(hotel_df, df2, left_index=True, right_index=True)
hotel_full

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,hotel,address,rating,user_ratings_total
0,biltine,14.525174,20.928701,Clear,78.44,20,1011,8.59,Hôtel Trois Étoiles de Biltine,"GWH7+2R8, Biltine",3.0,2.0
1,bargal,11.286823,51.077950,Clear,78.40,84,1013,5.12,Maka Hotel,"73QH+9F5, Bargaal",3.7,9.0
2,cayenne,4.937114,-52.325831,Clear,75.22,94,1012,1.14,Grand Hotel Montabo,Chemin St Hilaire - rue de Montabo,3.6,320.0
3,tsihombe,-25.318547,45.484057,Clear,73.17,86,1012,2.59,Littoral Hôtel,Tsihombe Tsihombe Androy,5.0,1.0
4,keshod,21.301130,70.252402,Clear,72.12,62,1013,7.18,"Hotel Shree Inn, KESHOD","Junagadh Road, near,iocl petrol pump, Keshod",3.7,332.0
5,nelson bay,-32.721814,152.144089,Clear,70.48,66,1021,1.99,Ramada Resort by Wyndham Shoal Bay,"35-45 Shoal Bay Rd, Shoal Bay",3.9,1013.0
6,lahij,13.057782,44.883614,Clear,76.03,80,1013,7.90,عمارة بن العطري,2W86+2J2، صبر،,2.8,5.0
7,riyadh,24.638916,46.716010,Clear,73.78,21,1012,5.59,Four Points by Sheraton Riyadh Khaldia,"Imam Faisal Bin Turki Street, Riyadh",4.1,4719.0
8,concepcion del oro,24.613705,-101.418787,Clear,72.72,36,1015,6.89,Hotel Hacienda San Javier,"Av. Morelos 15, El Grasero, Concepción del Oro",4.1,297.0
9,arlit,18.739079,7.393331,Clear,73.74,17,1015,9.26,Hôtel Temet,"P9RX+27R, Arlit",3.8,84.0


In [22]:
infos = []

for indx, row in hotel_full.iterrows():
    info_box_template = f"""<dl>
                        <dt>Hotel Name:</dt><dd>{row.hotel}</dd>
                        <dt>Rating:</dt><dd>{row.rating}</dd>
                        <dt>Address:</dt><dd>{row.address}</dd>
                        <dt>Location:</dt><dd>({row.latitude}, {row.longitude})</dd>
                        </dl>
                        """
    infos.append(info_box_template)
    
infos[0]

'<dl>\n                        <dt>Hotel Name:</dt><dd>Hôtel Trois Étoiles de Biltine</dd>\n                        <dt>Rating:</dt><dd>3.0</dd>\n                        <dt>Address:</dt><dd>GWH7+2R8, Biltine</dd>\n                        <dt>Location:</dt><dd>(14.525174, 20.928701)</dd>\n                        </dl>\n                        '

In [24]:
 # get data
data = df.loc[:, ["latitude", "longitude"]]
hotel_data = hotel_full.loc[:, ["latitude", "longitude"]]

# Create BASE, init the graph
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='SATELLITE')

# init the overlay layers
heat = gmaps.heatmap_layer(data, weights=df.humidity)
fig.add_layer(heat)

markers = gmaps.marker_layer(hotel_data, info_box_content=infos)
fig.add_layer(markers)

# show the plot
fig

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…

In [ ]:
# Add marker layer ontop of heat map


# Display figure
